# Portfolio Optimization

In [4]:
#!pip install cvxpy

In [11]:
import numpy as np
import pandas as pd
import pickle
import cvxpy as cp
import os

In [7]:
predictor = pd.read_csv('../Step3-Portfolio_Construction/predictor.csv')
predictor2 = pd.read_csv('../Step3-Portfolio_Construction/predictor2.csv')

In [9]:
predictor.head()

,Unnamed: 0,AAPL,AAP,AAL,ABC,ABBV,ABI,BVSN,AES,AMGN,...,VFC,VAR,VRTS,VRTX,VRSN,VRSK,VTR,VZ,VTSS,VSTNQ
0,1996-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1996-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1996-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1996-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1996-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Get Returns
rets = pd.read_hdf('../Step1-Data/1-rets.h5')
rets.tail()

,A,AABA,AAL,AAMRQ,AAP,AAPL,ABBV,ABC,ABI,ABKFQ,...,XRX,XTO,XYL,YNR,YRCW,YUM,ZBH,ZBRA,ZION,ZTS
2021-08-02,-0.001044,NaN,-0.015702,NaN,0.003065,-0.002331,-0.007309,0.004584,NaN,NaN,...,NaN,NaN,-0.001669,NaN,NaN,0.007611,-0.002509,-0.002733,-0.009971,-0.002911
2021-08-03,0.005161,NaN,-0.005483,NaN,0.012646,0.012644,0.006929,0.007985,NaN,NaN,...,NaN,NaN,0.018227,NaN,NaN,0.007780,-0.057052,-0.007677,0.021499,0.009846
2021-08-04,0.002665,NaN,-0.025063,NaN,-0.026602,-0.002782,-0.010065,0.014874,NaN,NaN,...,NaN,NaN,-0.009927,NaN,NaN,0.000750,-0.035652,0.024253,-0.009670,0.003381
2021-08-05,0.004991,NaN,0.075064,NaN,-0.015882,0.000749,0.000348,-0.029152,NaN,NaN,...,NaN,NaN,0.006790,NaN,NaN,0.000899,0.013088,0.016464,0.017040,-0.004639
2021-08-06,0.004257,NaN,0.005261,NaN,-0.003005,-0.004767,-0.005820,-0.010255,NaN,NaN,...,NaN,NaN,0.010038,NaN,NaN,0.002918,-0.007791,-0.012210,0.035203,-0.009615


Let the weight of our portfolio/assets be $w_i$ We want to optimize our predictors subject to the following constraints:

* Each asset has a weight of less than 3\%: $|w_i| < 0.03 ~\forall~ i$
* The absolute values of our weights need to sum to our leverage: $\sum_i |w_i| = 1$
* The weights need to sum to zero: $\sum_i w_i = 0$

We can put these into the cvxpy package and have it solve for our weights:

In [18]:
n = len(predictor.columns)
w = cp.Variable(n)
gamma = cp.Parameter(nonneg=True)
omega = rets.cov()

In [19]:
risk = cp.quad_form(w, omega)
prob = cp.Problem(cp.Maximize(ret - gamma*omega), 
               [cp.sum(w) == 1, 
                w >= 0])

Exception: Invalid dimensions for arguments.